In [32]:
# If you can't make this work with a new environment and the usual commands:
# conda install -c bokeh jupyter_bokeh
# conda install -c conda-forge hvplot panel geopandas-base plotly openpyxl fiona pyogrio

# Below is a workaround to install packages all accessible packages from the original conda_list.txt.
# Try the following code (it will take a while to run):

#import subprocess
#
#def pip_install(package):
#    try:
#        subprocess.check_call(['pip', 'install', package])
#    except subprocess.CalledProcessError:
#        print(f"Error installing package: {package}")
#
#def install_packages_from_requirements(requirements_file):
#    with open(requirements_file, 'r') as file:
#        for line in file:
#            package = line.strip()
#            if not package or package.startswith('#'):
#                continue
#            pip_install(package)
#
#if __name__ == '__main__':
#    requirements_file = 'requirements.txt'
#    install_packages_from_requirements(requirements_file)

In [33]:
from urllib.request import urlopen
import numpy as np
import pandas as pd
import geopandas as gpd
import panel as pn
import param
import hvplot.pandas
import plotly.express as px
import plotly.graph_objects as go
import json
from bokeh.resources import INLINE

In [34]:
def load_filtered_counties(url, target_states):
    with urlopen(url) as response:
        counties = json.load(response)
    counties['features'] = [f for f in counties['features'] if f['properties']['STATE'] in target_states]
    return counties

counties_url = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'
target_states = ['48']
counties = load_filtered_counties(counties_url, target_states)

dis_df = pd.read_csv("Data/District_Loss/district18-22.csv")
employ_df = pd.read_excel("Data/District_Loss/txcountyemploy.xlsx")
disloss_df = pd.read_csv("Data/District_Loss/DATA_Texas_District_v1.csv")

dis_copy = dis_df.copy()
countynames = dis_df['County Name'].unique()
countynames.sort()
fips = dis_df['County #'].unique()
fips.sort()
dis_copy = dis_copy.groupby('County Name').mean()
dis_copy = dis_copy.drop(['District #'], axis = 1)
dis_copy['County #'] = fips
dis_copy['County Name'] = countynames
dis_copy= dis_copy.reset_index(drop=True)

dis_copy['Full-Time Equivalent(FTE) Teachers Difference (2021-2019)'] = (dis_copy['FTE Teachers 2020-2021']-dis_copy['FTE Teachers 2018-2019'])/dis_copy['FTE Teachers 2018-2019']
dis_copy['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2019)'] = (dis_copy['Total Staff 2020-2021']-dis_copy['Total Staff 2018-2019'])/dis_copy['Total Staff 2018-2019']
dis_copy['Total Number of Students Difference (2021-2019)'] = (dis_copy['Total Students 2020-2021']-dis_copy['Total Students 2018-2019'])/dis_copy['Total Students 2018-2019']

dis_copy['Full-Time Equivalent(FTE) Teachers Difference (2021-2022)'] = (dis_copy['FTE Teachers 2021-2022']-dis_copy['FTE Teachers 2020-2021'])/dis_copy['FTE Teachers 2020-2021']
dis_copy['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2022)'] = (dis_copy['Total Staff 2021-2022']-dis_copy['Total Staff 2020-2021'])/dis_copy['Total Staff 2020-2021']
dis_copy['Total Number of Students Difference (2021-2022)'] = (dis_copy['Total Students 2021-2022']-dis_copy['Total Students 2020-2021'])/dis_copy['Total Students 2020-2021']

dis_copy['Full-Time Equivalent(FTE) Teachers Difference (2019-2022)'] = (dis_copy['FTE Teachers 2021-2022']-dis_copy['FTE Teachers 2018-2019'])/dis_copy['FTE Teachers 2018-2019']
dis_copy['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2022)'] = (dis_copy['Total Staff 2021-2022']-dis_copy['Total Staff 2018-2019'])/dis_copy['Total Staff 2018-2019']
dis_copy['Total Number of Students Difference (2019-2022)'] = (dis_copy['Total Students 2021-2022']-dis_copy['Total Students 2018-2019'])/dis_copy['Total Students 2018-2019']

dis_copy = dis_copy.round(2)

employ_df['NAME'] = employ_df['NAME'].str.replace(", TX","")
employ_df['YEAR'] = employ_df['YEAR'].values.astype(str)
employ_df['YEAR'] = employ_df['YEAR'].str.replace("'","")
df_19 = employ_df[employ_df['YEAR'] == '2019']
df_20 = employ_df[employ_df['YEAR'] == '2020']
df_21 = employ_df[employ_df['YEAR'] == '2021']
df_20 = df_20.reset_index()
df_21 = df_21.reset_index()
diff_df = pd.DataFrame()
diff_df = df_19[['NAME','FIPS']]
diff_df['Civilian Labor Force Difference (2021-2019)'] = (df_21['LABOR_FORCE']-df_19['LABOR_FORCE'])/df_19['LABOR_FORCE']
diff_df['Employed Persons Difference (2021-2019)'] = (df_21['EMPLOYED']-df_19['EMPLOYED'])/df_19['EMPLOYED']
diff_df['Umemployed Persons Difference (2021-2019)'] = (df_21['UNEMPLOYED']-df_19['UNEMPLOYED'])/df_19['UNEMPLOYED']
diff_df['Unemployment Rate Difference (2021-2019)'] = (df_21['UNEMPLOYMENT_PCT']-df_19['UNEMPLOYMENT_PCT'])/df_19['UNEMPLOYMENT_PCT']

diff_df = diff_df.round(2)
diff_df.rename(columns={'NAME':'County Name','FIPS':'County #'},inplace=True)



disloss_df['County #'] = disloss_df['County #'].astype(int)
countynames = dis_df['County Name'].unique()
countynames.sort()
fips = dis_df['County #'].unique()
fips.sort()

c:\Users\Leaf\anaconda3\envs\EducationDashboard\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

C:\Users\Leaf\AppData\Local\Temp\ipykernel_23360\459414375.py:20: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\Leaf\AppData\Local\Temp\ipykernel_23360\459414375.py:50: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Leaf\AppData\Local\Temp\ipykernel_23360\459414375.py:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [35]:
# This is the cell we change to switch from percentages to raw numbers
discopy_df = pd.DataFrame()
discopy_df['County #'] = disloss_df['County #']

# Percentages Version
#discopy_df['Average Score Difference(%) in Math (2021-2019)'] = (disloss_df['AVG_Math_2021']-disloss_df['AVG_Math_2019'])/disloss_df['AVG_Math_2019']
#discopy_df['Average Score Difference(%) in Reading (2021-2019)'] = (disloss_df['AVG_Reading_2021']-disloss_df['AVG_Reading_2019'])/disloss_df['AVG_Reading_2019']
#discopy_df['Average Score Difference(%) in Math (2022-2021)'] = (disloss_df['AVG_Math_2022']-disloss_df['AVG_Math_2021'])/disloss_df['AVG_Math_2021']
#discopy_df['Average Score Difference(%) in Reading (2022-2021)'] = (disloss_df['AVG_Reading_2022']-disloss_df['AVG_Reading_2021'])/disloss_df['AVG_Reading_2021']
#discopy_df['Average Score Difference(%) in Math (2022-2019)'] = (disloss_df['AVG_Math_2022']-disloss_df['AVG_Math_2019'])/disloss_df['AVG_Math_2019']
#discopy_df['Average Score Difference(%) in Reading (2022-2019)'] = (disloss_df['AVG_Reading_2022']-disloss_df['AVG_Reading_2019'])/disloss_df['AVG_Reading_2019']

# Raw Numbers Version
discopy_df['Average Score Difference(%) in Math (2021-2019)'] = (disloss_df['AVG_Math_2021']-disloss_df['AVG_Math_2019'])
discopy_df['Average Score Difference(%) in Reading (2021-2019)'] = (disloss_df['AVG_Reading_2021']-disloss_df['AVG_Reading_2019'])
discopy_df['Average Score Difference(%) in Math (2022-2021)'] = (disloss_df['AVG_Math_2022']-disloss_df['AVG_Math_2021'])
discopy_df['Average Score Difference(%) in Reading (2022-2021)'] = (disloss_df['AVG_Reading_2022']-disloss_df['AVG_Reading_2021'])
discopy_df['Average Score Difference(%) in Math (2022-2019)'] = (disloss_df['AVG_Math_2022']-disloss_df['AVG_Math_2019'])
discopy_df['Average Score Difference(%) in Reading (2022-2019)'] = (disloss_df['AVG_Reading_2022']-disloss_df['AVG_Reading_2019'])

discopy_df = discopy_df.groupby('County #').mean()
discopy_df['County #'] = fips
discopy_df['County Name'] = countynames
discopy_df = discopy_df.reset_index(drop=True)

discopy_df = discopy_df.round(2)

discopy_df.drop(columns='County Name',inplace=True)
dis_copy.drop(columns='County Name',inplace=True)
comp_df = discopy_df.merge(diff_df,on='County #')
comp_df = comp_df.merge(dis_copy,on='County #')

# Texas County Level Map

In [36]:
column_mapping = {
    'Full-Time Equivalent(FTE) Teachers Difference (2021-2019)': 'Number of Teachers (2019-2021)',
    'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2019)': 'Number of Overall Staff (2019-2021)',
    'Total Number of Students Difference (2021-2019)': 'Number of Students (2019-2021)',
    'Civilian Labor Force Difference (2021-2019)': 'Labor Force (2019-2021)',    

    'Average Score Difference(%) in Math (2021-2019)': 'STAAR Score in Math (2019-2021)',
    'Average Score Difference(%) in Reading (2021-2019)': 'STAAR Score in Reading (2019-2021)',
    'Average Score Difference(%) in Math (2022-2021)': 'STAAR Score in Math (2021-2022)',
    'Average Score Difference(%) in Reading (2022-2021)': 'STAAR Score in Reading (2021-2022)',
    'Average Score Difference(%) in Math (2022-2019)': 'STAAR Score in Math (2019-2022)',
    'Average Score Difference(%) in Reading (2022-2019)': 'STAAR Score in Reading (2019-2022)',

    'Full-Time Equivalent(FTE) Teachers Difference (2021-2022)':'Number of Teachers (2021-2022)',
    'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2022)': 'Number of Overall Staff (2021-2022)',
    'Total Number of Students Difference (2021-2022)': 'Number of Students (2021-2022)',

    'Full-Time Equivalent(FTE) Teachers Difference (2019-2022)':'Number of Teachers (2019-2022)',
    'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2022)': 'Number of Overall Staff (2019-2022)',
    'Total Number of Students Difference (2019-2022)': 'Number of Students (2019-2022)'}
    
comp_df.rename(columns=column_mapping, inplace=True)

multiplier_columns = [
    'Number of Teachers (2019-2021)', 
    'Number of Overall Staff (2019-2021)',
    'Number of Students (2019-2021)', 
    'Labor Force (2019-2021)',
    'Number of Teachers (2021-2022)', 
    'Number of Overall Staff (2021-2022)',
    'Number of Students (2021-2022)', 
    'Number of Teachers (2019-2022)', 
    'Number of Overall Staff (2019-2022)',
    'Number of Students (2019-2022)', 
    'STAAR Score in Math (2019-2021)',
    'STAAR Score in Reading (2019-2021)', 
    'STAAR Score in Math (2021-2022)',
    'STAAR Score in Reading (2021-2022)',
    'STAAR Score in Math (2019-2022)',
    'STAAR Score in Reading (2019-2022)'
]

comp_df[multiplier_columns] *= 100

comp_df['County Name'] = comp_df['County Name'].str[:-7]

score_columns = [
    'STAAR Score in Math (2021-2022)', 'STAAR Score in Reading (2021-2022)',
    'STAAR Score in Math (2019-2022)', 'STAAR Score in Reading (2019-2022)',
    'STAAR Score in Math (2019-2021)', 'STAAR Score in Reading (2019-2021)'
]

comp_df[score_columns] = comp_df[score_columns].astype(int)

year_columns = [
    'Number of Teachers (2021-2022)', 'Number of Overall Staff (2021-2022)',
    'Number of Students (2021-2022)', 'Number of Teachers (2019-2022)',
    'Number of Overall Staff (2019-2022)', 'Number of Students (2019-2022)',
    'Number of Teachers (2019-2021)', 'Number of Overall Staff (2019-2021)',
    'Number of Students (2019-2021)'
]

comp_df[year_columns] = comp_df[year_columns].astype(int)

df1921 = comp_df.rename(columns={
    'Number of Teachers (2019-2021)': 'Number of Teachers',
    'Number of Overall Staff (2019-2021)': 'Number of Staff',
    'Number of Students (2019-2021)': 'Number of Students',
    'STAAR Score in Math (2019-2021)': 'STAAR Score in Math',
    'STAAR Score in Reading (2019-2021)': 'STAAR Score in Reading',
    'Labor Force (2019-2021)': 'Labor Force'
})[['County Name', 'County #', 'Number of Teachers', 'Number of Staff',
    'Number of Students', 'Labor Force', 'STAAR Score in Math', 'STAAR Score in Reading']]

df2122 = comp_df.rename(columns={
    'Number of Teachers (2021-2022)': 'Number of Teachers',
    'Number of Overall Staff (2021-2022)': 'Number of Staff',
    'Number of Students (2021-2022)': 'Number of Students',
    'STAAR Score in Math (2021-2022)': 'STAAR Score in Math',
    'STAAR Score in Reading (2021-2022)': 'STAAR Score in Reading'
})[['County Name', 'County #', 'Number of Teachers', 'Number of Staff',
    'Number of Students', 'STAAR Score in Math', 'STAAR Score in Reading']]

df1922 = comp_df.rename(columns={
    'Number of Teachers (2019-2022)': 'Number of Teachers',
    'Number of Overall Staff (2019-2022)': 'Number of Staff',
    'Number of Students (2019-2022)': 'Number of Students',
    'STAAR Score in Math (2019-2022)': 'STAAR Score in Math',
    'STAAR Score in Reading (2019-2022)': 'STAAR Score in Reading'
})[['County Name', 'County #', 'Number of Teachers', 'Number of Staff',
    'Number of Students', 'STAAR Score in Math', 'STAAR Score in Reading']]

In [37]:
# Why are we exporting this data?
# comp_df.to_csv('Data/CountyLevelstats.csv')

In [38]:
# Creates a county level map of Texas based on the Texas Education Agency Data
input_list = ['STAAR Score in Math',
              'STAAR Score in Reading',
              'Number of Students',
              'Number of Teachers',
              'Number of Staff']
# Labor Force Taken out for now 2019-2021
year_list = ['2019-2021','2021-2022','2019-2022']
class MapDashboard(param.Parameterized):
    _  = param.Selector(input_list,default=input_list[0])
    __ = param.Selector(default=year_list[0],objects=year_list)

    def get_map(self):
        if self.__ == '2019-2021':
            data = df1921[['County Name','County #',self._]]
        elif self.__ == '2021-2022':
            data = df2122[['County Name','County #',self._]]
        elif self.__ == '2019-2022':
            data = df1922[['County Name','County #',self._]]
        fig = px.choropleth_mapbox(data, geojson=counties, locations='County #', color= self._,
        color_continuous_scale='balance_r',
        range_color=(-10,10),
        center = {'lat':31.9686, 'lon': -99.9018},
        zoom = 4.65,
        height=585, width=525,
        mapbox_style = "white-bg",       
        custom_data = [data['County Name'],self._]
        )
        hovertemp = ''
        hovertemp = '%{customdata[0]} County<br>' 
        # Quick if/then function to check if the hovertext should be absolute or percentage
        if 'STAAR Score' in self._:
            hovertemp += '%{customdata[1]} point change for ' + self._
        else:
            hovertemp += '%{customdata[1]}% change in ' + self._
        fig.update_traces(hovertemplate=hovertemp)
        fig.update_geos(fitbounds='locations')
        fig.update_layout(hoverlabel=dict(bgcolor='black',font_size=24,font_family="Times New Roman"),
        font=dict(family='times new roman',
        size=24,color='black'),                
        margin={"r":10,"t":0,"l":10,"b":0})
        fig.update_layout(coloraxis_colorbar=dict(tickvals=[-18,-9,0,9,18],ticktext=['-20','-10','0','10','20'])),
        fig.update_coloraxes(cmid=0,colorbar_orientation='h',colorbar_y=-.148,colorbar_title_font_family='Times New Roman',
                             colorbar_title_font_size=20,colorbar_title_side='bottom', colorbar_thickness=4)
        return fig

    def get_description(self):
        descriptions = {
            "2019-2021": {
                "STAAR Score in Math": "The total difference in <b>Math STAAR scores</b> between <b>2019 and 2021</b> for a district.",
                "STAAR Score in Reading": "The total difference in <b>Reading STAAR scores</b> between <b>2019 and 2021</b> for a district.",
                "Number of Teachers": "The percentage difference in <b>Full-Time Teachers</b> between <b>2019 and 2021</b> for a district.",
                "Number of Staff": "The percentage difference in <b>staff</b> between <b>2019 and 2021</b> for a district.",
                "Number of Students": "The percentage difference in <b>students</b> between <b>2019 and 2021</b> for a district."
            },
            "2021-2022": {
                "STAAR Score in Math": "The total difference in <b>Math STAAR scores</b> between <b>2021 and 2022</b> for a district.",
                "STAAR Score in Reading": "The total difference in <b>Reading STAAR scores</b> between <b>2021 and 2022</b> for a district.",
                "Number of Teachers": "The percentage difference in <b>Full-Time Teachers</b> between <b>2021 and 2022</b> for a district.",
                "Number of Staff": "The percentage difference in <b>staff</b> between <b>2021 and 2022</b> for a district.",
                "Number of Students": "The percentage difference in <b>students</b> between <b>2021 and 2022</b> for a district."
            },
            "2019-2022": {
                "STAAR Score in Math": "The total difference in <b>Math STAAR scores</b> between <b>2019 and 2022</b> for a district.",
                "STAAR Score in Reading": "The total difference in <b>Reading STAAR scores</b> between <b>2019 and 2022</b> for a district.",
                "Number of Teachers": "The percentage difference in <b>Full-Time Teachers</b> between <b>2019 and 2022</b> for a district.",
                "Number of Staff": "The percentage difference in <b>staff</b> between <b>2019 and 2022</b> for a district.",
                "Number of Students": "The percentage difference in <b>students</b> between <b>2019 and 2022</b> for a district."
            },
        }

        if self.__ in descriptions:
            description_text = descriptions[self.__].get(self._)
            if description_text:
                return pn.pane.HTML(description_text, 
                                    styles={'color': 'black', 'font-size': '20px', 'width': '125%',
                                            'text-align': 'center', 'max-width': '500px', 'word-wrap': 'break-word',
                                            'margin-left': '147px', 'font-family': 'Times New Roman'})
        return description_pane

    def panel(self):
        return pn.Row(
            pn.Column(
            pn.pane.HTML("""<h1>Select A Variable To View At The County Level</h1>""",
                              styles={'color':'black','font-size':'14px','width':'100%',
                                     'text-align':'center',
                                     'font-family':'Times New Roman'}),
            pn.Param(self,width=550,styles={'background':'#FFFFFF','margin-left':'225px'},
            widgets={'__':{'widget_type':pn.widgets.RadioButtonGroup}}),
            pn.Spacer(height=50),
            pn.pane.HTML("""<h1>What does this variable show us?</h1>""",
                            styles={'color':'black','font-size':'14px','width':'100%',
                                  'text-align':'center',
                                  'font-family':'Times New Roman'}),
            pn.Spacer(height= 5),
            self.get_description
            ),
            pn.Spacer(width=150),
            self.get_map
        )
dashboard = MapDashboard(name='')
map_component1 = dashboard.panel()
map_component1.save('CountyLevel_Tab',resources=INLINE)

# Heatmap & Histogram

In [39]:
address = "Data/District_Loss/DATA_Texas_District_v1.csv"
loss_df = pd.read_csv(address)

# scale columns to be percentages
columns_to_scale = ['% LOSS Math 2019 to 2022', '% LOSS Reading 2019 to 2022',
                    '% LOSS Math 2021 to 2022', '% LOSS Reading 2021 to 2022',
                    '% LOSS Math 2019 to 2021', '% LOSS Reading 2019 to 2021']

loss_df[columns_to_scale] *= 100
loss_df = loss_df.round(2)

In [40]:
# creates a histogram of the normalized difference in STAAR scores
input_dict = {
    '2019-2021': {'Math': '% LOSS Math 2019 to 2021', 'Reading': '% LOSS Reading 2019 to 2021'},
    '2021-2022': {'Math': '% LOSS Math 2021 to 2022', 'Reading': '% LOSS Reading 2021 to 2022'},
    '2019-2022': {'Math': '% LOSS Math 2019 to 2022', 'Reading': '% LOSS Reading 2019 to 2022'}
}

class LossDashboard(param.Parameterized):
    _ = param.Selector(list(input_dict.keys()), default=list(input_dict.keys())[0])

    def get_map(self):
        fig = go.Figure()

        for subject, column in input_dict[self._].items():
            fig.add_trace(go.Histogram(x=loss_df[column], name=subject))

        fig.update_layout(barmode='stack', xaxis_range=[-30, 30], xaxis_dtick=10,
                          yaxis_range=[0, 600], yaxis_dtick=100,
                          height=500, width=680,
                          xaxis_title_text='% Loss', yaxis_title_text='Number of Districts',
                          bargap=0.2, hovermode=False,
                          hoverlabel=dict(bgcolor='black', font_size=24, font_family="Times New Roman"),
                          font=dict(family='times new roman', size=23, color='black'),
                          title_x=0.55,
                          margin={"r": 10, "t": 10, "l": 0, "b": 15})

        return fig

    def panel(self):
        return pn.Column(
            pn.Param(self, width=450, styles={'background': '#FFFFFF'},
                     widgets={'_': {'widget_type': pn.widgets.RadioButtonGroup}}),
            self.get_map
        )

dashboard = LossDashboard(name='Normalized Difference in STAAR Scores, Avg. Over TX ISDs')
loss_component1 = dashboard.panel()
loss_component1.save('Histogram_Tab', resources=INLINE)


In [41]:
# Refactored from original discrete operations, changed to functional and vectorized operations.
heat_df = pd.read_csv("Data/District_Loss/heat_df.csv")
heat_df = heat_df.set_index('group')
heat_df[['3.0', '4.0', '5.0', '6.0', '7.0', '8.0']] *= -100
heat_df = heat_df.round(2)
heat_df.rename(columns={'3.0': '3rd', '4.0': '4th', '5.0': '5th', '6.0': '6th', '7.0': '7th', '8.0': '8th'}, inplace=True)

# Process heat1921, heat1922, and heat2122
years = ['1921', '1922', '2122']
for year in years:
    heat_data = pd.read_excel(f"Data/District_Loss/heat{year}.xlsx")
    heat_data = heat_data.set_index('group')
    heat_data[['3rd', '4th', '5th', '6th', '7th', '8th']] *= 100
    heat_data = heat_data.round(2)
    globals()[f'heat{year}'] = heat_data

In [42]:
subject_inputs = ['Math', 'Reading']
year_list = ['2019-2021', '2021-2022', '2019-2022']

class LossDashboard(param.Parameterized):
    Subject = param.Selector(subject_inputs, default=subject_inputs[0])
    Year = param.Selector(year_list, default=year_list[0])

    def get_data(self):
        data_dict = {
            '2019-2021': heat1921,
            '2019-2022': heat1922,
            '2021-2022': heat2122
        }
        data = data_dict[self.Year][data_dict[self.Year]['subject'] == self.Subject]
        return data.drop(['subject'], axis=1)

    @param.depends('Subject', 'Year')
    def lossplot(self):
        data = self.get_data()
        fig = px.imshow(data, labels=dict(x='Grade', y='Demographic Group', color='Learning Loss %'), text_auto='.1f',
                        height=450, width=800,
                        color_continuous_scale='balance_r',
                        range_color=(-5, 5),
                        aspect='none')
        fig.update_layout(
            hoverlabel=dict(bgcolor='black', font_size=22, font_family="Times New Roman"),
            font=dict(family='times new roman', size=22, color='black'),
            title_x=.96,
            margin={"r": 10, "t": 10, "l": 0, "b": 15}
        )
        fig.update_coloraxes(cmid=0, colorbar_title_font_family='Times New Roman', colorbar_title_text='% Loss',
                             colorbar_title_font_size=22, colorbar_title_side='bottom', colorbar_thickness=10)
        return fig

    def panel(self):
        return pn.Column(
            pn.Param(self, width=550, styles={'background': '#FFFFFF'}, widgets={'Year': pn.widgets.RadioButtonGroup, 'Subject': pn.widgets.RadioButtonGroup}),
            self.lossplot
        )

dashboard = LossDashboard(name='Normalized Difference in STAAR Scores By Demographic Group and Grade')
loss_component2 = dashboard.panel()
loss_component2

BokehModel(combine_events=True, render_bundle={'docs_json': {'68a73932-6eac-4e69-9364-355aa1c7850d': {'version…

# Texas District Level Maps

In [43]:
tx_map = gpd.read_file("Data/ISDmaps/Districts2020to2021.geojson")
label_data = pd.read_csv("Data/District_Loss/DATA_Texas_District_v1.csv")

In [44]:
label_data["NCES_DISTR"] = label_data["NCES_DISTR"].astype(int)
label_data = label_data.sort_values("NCES_DISTR")
tx_map = tx_map.sort_values("NCES_DISTR")
tx_map['NCES_DISTR'] = tx_map['NCES_DISTR'].astype(int)

In [45]:
merged_df = tx_map.merge(label_data, how = 'inner', on=['NCES_DISTR','DISTRICT_N'])
merged_df['Full-Time Equivalent(FTE) Teachers Difference (2019-2021)'] = (merged_df['FTE Teachers 2020-2021']-merged_df['FTE Teachers 2018-2019'])/merged_df['FTE Teachers 2018-2019']
merged_df['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2021)'] = (merged_df['Total Staff 2020-2021']-merged_df['Total Staff 2018-2019'])/merged_df['Total Staff 2018-2019']
merged_df['Total Number of Students Difference (2019-2021)'] = (merged_df['Total Students 2020-2021']-merged_df['Total Students 2018-2019'])/merged_df['Total Students 2018-2019']

merged_df['Full-Time Equivalent(FTE) Teachers Difference (2021-2022)'] = (merged_df['FTE Teachers 2021-2022']-merged_df['FTE Teachers 2020-2021'])/merged_df['FTE Teachers 2020-2021']
merged_df['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2022)'] = (merged_df['Total Staff 2021-2022']-merged_df['Total Staff 2020-2021'])/merged_df['Total Staff 2020-2021']
merged_df['Total Number of Students Difference (2021-2022)'] = (merged_df['Total Students 2021-2022']-merged_df['Total Students 2020-2021'])/merged_df['Total Students 2020-2021']

merged_df['Full-Time Equivalent(FTE) Teachers Difference (2019-2022)'] = (merged_df['FTE Teachers 2021-2022']-merged_df['FTE Teachers 2018-2019'])/merged_df['FTE Teachers 2018-2019']
merged_df['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2022)'] = (merged_df['Total Staff 2021-2022']-merged_df['Total Staff 2018-2019'])/merged_df['Total Staff 2018-2019']
merged_df['Total Number of Students Difference (2019-2022)'] = (merged_df['Total Students 2021-2022']-merged_df['Total Students 2018-2019'])/merged_df['Total Students 2018-2019']

merged_df = merged_df.round(2)

In [46]:
# Vectorized renames
rename_mappings = {
    '% LOSS All 2019 to 2021': 'Average Score Difference(%) in All (2019-2021)',
    '% LOSS All 2019 to 2022': 'Average Score Difference(%) in All (2019-2022)',
    '% LOSS All 2021 to 2022': 'Average Score Difference(%) in All (2021-2022)',
    '% LOSS Math 2019 to 2021': 'Average Score Difference(%) in Math (2019-2021)',
    '% LOSS Math 2019 to 2022': 'Average Score Difference(%) in Math (2019-2022)',
    '% LOSS Math 2021 to 2022': 'Average Score Difference(%) in Math (2021-2022)',
    '% LOSS Reading 2019 to 2021': 'Average Score Difference(%) in Reading (2019-2021)',
    '% LOSS Reading 2019 to 2022': 'Average Score Difference(%) in Reading (2019-2022)',
    '% LOSS Reading 2021 to 2022': 'Average Score Difference(%) in Reading (2021-2022)',
    'Full-Time Equivalent(FTE) Teachers Difference (2019-2021)': 'Number of Teachers (2019-2021)',
    'Full-Time Equivalent(FTE) Teachers Difference (2019-2022)': 'Number of Teachers (2019-2022)',
    'Full-Time Equivalent(FTE) Teachers Difference (2021-2022)': 'Number of Teachers (2021-2022)',
    'Label_All (19-21)': 'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2019-2021)',
    'Label_All (21-22)': 'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2021-2022)',
    'Label_All (19-22)': 'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2019-2022)',
    'Label_Math (19-21)': 'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2019-2021)',
    'Label_Math (19-22)': 'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2019-2022)',
    'Label_Math (21-22)': 'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2021-2022)',
    'Label_Reading (19-21)': 'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2019-2021)',
    'Label_Reading (19-22)': 'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2019-2022)',
    'Label_Reading (21-22)': 'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2021-2022)',
    'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2021)': 'Number of Overall Staff (2019-2021)',
    'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2022)': 'Number of Overall Staff (2019-2022)',
    'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2022)': 'Number of Overall Staff (2021-2022)',
    'Total Number of Students Difference (2019-2021)': 'Number of Students (2019-2021)',
    'Total Number of Students Difference (2019-2022)': 'Number of Students (2019-2022)',
    'Total Number of Students Difference (2021-2022)': 'Number of Students (2021-2022)',
    'Average Score Difference(%) in Math (2019-2021)': 'STAAR Score in Math (2019-2021)',
    'Average Score Difference(%) in Math (2019-2022)': 'STAAR Score in Math (2019-2022)',
    'Average Score Difference(%) in Math (2021-2022)': 'STAAR Score in Math (2021-2022)',
    'Average Score Difference(%) in Reading (2019-2021)': 'STAAR Score in Reading (2019-2021)',
    'Average Score Difference(%) in Reading (2019-2022)': 'STAAR Score in Reading (2019-2022)',
    'Average Score Difference(%) in Reading (2021-2022)': 'STAAR Score in Reading (2021-2022)',
    'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2019-2022)': 'Learning Loss in Math (2019-2022)',
    'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2021-2022)': 'Learning Loss in Math (2021-2022)',
    'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2019-2021)': 'Learning Loss in Math (2019-2021)',
    'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2019-2022)': 'Learning Loss in Reading (2019-2022)',
    'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2021-2022)': 'Learning Loss in Reading (2021-2022)',
    'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2019-2021)': 'Learning Loss in Reading (2019-2021)',
    'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2019-2022)': 'Learning Loss (2019-2022)',
    'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2021-2022)': 'Learning Loss (2021-2022)',
    'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2019-2021)': 'Learning Loss (2019-2021)',
}

# Running command twice is a temporary fix due to columns created first pass then renamed second pass.
merged_df.rename(columns=rename_mappings, inplace=True)
merged_df.rename(columns=rename_mappings, inplace=True)

# Vectorized multiplication
columns_to_multiply = [
    'Number of Overall Staff (2019-2021)',
    'Number of Overall Staff (2019-2022)',
    'Number of Overall Staff (2021-2022)',
    'Number of Students (2019-2021)',
    'Number of Students (2019-2022)',
    'Number of Students (2021-2022)',
    'Number of Teachers (2019-2021)',
    'Number of Teachers (2019-2022)',
    'Number of Teachers (2021-2022)',
    'STAAR Score in Math (2019-2021)',
    'STAAR Score in Math (2019-2022)',
    'STAAR Score in Math (2021-2022)',
    'STAAR Score in Reading (2019-2021)',
    'STAAR Score in Reading (2019-2022)',
    'STAAR Score in Reading (2021-2022)',
]

merged_df[columns_to_multiply] *= 100

# Add the learning loss columns
column_mapping = {
    'Loss% (2019-2021)': 'Learning Loss (2019-2021)',
    'Loss% (2019-2022)': 'Learning Loss (2019-2022)',
    'Loss% (2021-2022)': 'Learning Loss (2021-2022)',
    'Math% (2019-2021)': 'Learning Loss in Math (2019-2021)',
    'Math% (2019-2022)': 'Learning Loss in Math (2019-2022)',
    'Math% (2021-2022)': 'Learning Loss in Math (2021-2022)',
    'Reading% (2019-2021)': 'Learning Loss in Reading (2019-2021)',
    'Reading% (2019-2022)': 'Learning Loss in Reading (2019-2022)',
    'Reading% (2021-2022)': 'Learning Loss in Reading (2021-2022)',
}
# ** unpacks the dictionary into keyword arguments, which is what assign() expects.
merged_df = merged_df.assign(**column_mapping)

In [47]:
# Shortening the code using map function to vectorize.
cdict = comp_df.set_index('County Name')['County #'].to_dict()
ndict = comp_df.set_index('County #')['County Name'].to_dict()
label_data['County #'] = label_data['County #'].map(ndict)
label_data.rename(columns={'County #': 'County Name', 'County Number': 'County #'}, inplace=True)

In [48]:
print(cdict)

{'Anderson': 48001, 'Andrews': 48003, 'Angelina': 48005, 'Aransas': 48007, 'Archer': 48009, 'Armstrong': 48011, 'Atascosa': 48013, 'Austin': 48015, 'Bailey': 48017, 'Bandera': 48019, 'Bastrop': 48021, 'Baylor': 48023, 'Bee': 48025, 'Bell': 48027, 'Bexar': 48029, 'Blanco': 48031, 'Borden': 48033, 'Bosque': 48035, 'Bowie': 48037, 'Brazoria': 48039, 'Brazos': 48041, 'Brewster': 48043, 'Briscoe': 48045, 'Brooks': 48047, 'Brown': 48049, 'Burleson': 48051, 'Burnet': 48053, 'Caldwell': 48055, 'Calhoun': 48057, 'Callahan': 48059, 'Cameron': 48061, 'Camp': 48063, 'Carson': 48065, 'Cass': 48067, 'Castro': 48069, 'Chambers': 48071, 'Cherokee': 48073, 'Childress': 48075, 'Clay': 48077, 'Cochran': 48079, 'Coke': 48081, 'Coleman': 48083, 'Collin': 48085, 'Collingsworth': 48087, 'Colorado': 48089, 'Comal': 48091, 'Comanche': 48093, 'Concho': 48095, 'Cooke': 48097, 'Coryell': 48099, 'Cottle': 48101, 'Crane': 48103, 'Crockett': 48105, 'Crosby': 48107, 'Culberson': 48109, 'Dallam': 48111, 'Dallas': 4811

In [49]:
#is there any reason we are recalculating the data every pass? Setting to static unless this is part of the pipeline.

# county_dict = pd.DataFrame()
# county_dict['County #'] = comp_df['County #']
# county_dict['County Name'] = comp_df['County Name']
# county_dict.to_csv('countyfipsmatcher.csv')

In [50]:
subject_inputs = ['Math', 'Reading']
year_list = ['2019-2021', '2021-2022', '2019-2022']

class LossDashboard(param.Parameterized):
    Subject = param.Selector(subject_inputs, default=subject_inputs[0])
    Year = param.Selector(year_list, default=year_list[0])

    def get_data(self):
        data_dict = {
            '2019-2021': heat1921,
            '2019-2022': heat1922,
            '2021-2022': heat2122
        }
        data = data_dict[self.Year][data_dict[self.Year]['subject'] == self.Subject]
        return data.drop(['subject'], axis=1)

    @param.depends('Subject', 'Year')
    def lossplot(self):
        data = self.get_data()
        fig = px.imshow(data, labels=dict(x='Grade', y='Demographic Group', color='Learning Loss %'), text_auto='.1f',
                        height=450, width=800,
                        color_continuous_scale='balance_r',
                        range_color=(-5, 5),
                        aspect='none')
        fig.update_layout(
            hoverlabel=dict(bgcolor='black', font_size=22, font_family="Times New Roman"),
            font=dict(family='times new roman', size=22, color='black'),
            title_x=.96,
            margin={"r": 10, "t": 10, "l": 0, "b": 15}
        )
        fig.update_coloraxes(cmid=0, colorbar_title_font_family='Times New Roman', colorbar_title_text='% Loss',
                             colorbar_title_font_size=22, colorbar_title_side='bottom', colorbar_thickness=10)
        return fig

    def panel(self):
        return pn.Column(
            pn.Param(self, width=550, styles={'background': '#FFFFFF'}, widgets={'Year': pn.widgets.RadioButtonGroup, 'Subject': pn.widgets.RadioButtonGroup}),
            self.lossplot
        )

dashboard = LossDashboard(name='Normalized Difference in STAAR Scores By Demographic Group and Grade')
loss_component2 = dashboard.panel()
loss_component2

BokehModel(combine_events=True, render_bundle={'docs_json': {'9fbe5f17-e58d-4a5b-a1a8-9a8a363c526a': {'version…

In [51]:
merged_df['Learning Loss (2021-2022)']

0       0
1       1
2       0
3       1
4       1
       ..
1008    0
1009   -1
1010    0
1011    0
1012   -1
Name: Learning Loss (2021-2022), Length: 1013, dtype: int64

In [52]:
def generate_label(row):
    if row['Learning Loss'] > 0:
        return 'Gain'
    elif row['Learning Loss'] < 0:
        return 'Loss'
    else:
        return 'No Unexpected Change'

year_columns = ['2019-2021', '2021-2022', '2019-2022']
for year in year_columns:
    math_column = f'Learning Loss in Math ({year})'
    reading_column = f'Learning Loss in Reading ({year})'
    overall_column = f'Learning Loss ({year})'

    merged_df[f'Label Math ({year})'] = merged_df[math_column].apply(lambda x: generate_label({'Learning Loss': x}))
    merged_df[f'Label Reading ({year})'] = merged_df[reading_column].apply(lambda x: generate_label({'Learning Loss': x}))
    merged_df[f'Label Overall ({year})'] = merged_df[overall_column].apply(lambda x: generate_label({'Learning Loss': x}))

In [53]:
#merged_df.to_csv('Data/DistrictLevelstats.csv')

In [54]:
# Convert columns to integers
year_columns = ['2019-2021', '2021-2022', '2019-2022']
columns_to_convert = ['Number of Teachers', 'Number of Overall Staff', 'Number of Students',
                      'STAAR Score in Math', 'STAAR Score in Reading']

for year in year_columns:
    for column in columns_to_convert:
        col_name = f"{column} ({year})"
        merged_df[col_name] = merged_df[col_name].astype(int)


# Create DataFrames for each year
column_lists = [[f"{column} ({year})" for column in columns_to_convert] + ['Math% (2019-2021)', 'Reading% (2019-2021)', 'Loss% (2019-2021)']
                for year in year_columns]
df_1921, df_2122, df_1922 = [merged_df[['NAME', 'County #', 'DISTRICT_N'] + columns].copy()
                             for columns in column_lists]

# Rename columns
renaming_dict = {f"{column} ({year})": column for column in columns_to_convert for year in year_columns}
renaming_dict.update({'Math% (2019-2021)': 'Math%', 'Reading% (2019-2021)': 'Reading%', 'Loss% (2019-2021)': 'Loss%',
                      'Number of Overall Staff (2019-2022)': 'Number of Staff',
                      'Number of Overall Staff (2021-2022)': 'Number of Staff',
                      'Number of Overall Staff (2019-2021)': 'Number of Staff'})
df_1921.rename(columns=renaming_dict, inplace=True)
df_2122.rename(columns=renaming_dict, inplace=True)
df_1922.rename(columns=renaming_dict, inplace=True)

# Calculate 'Total Funding' and 'Funding Per Student'
funding_columns = ['CARES ESSER I 20', 'CRRSA ESSER II 21', 'ARP ESSER III 21', 'ESSER-SUPP 22', 'ESSER-SUPP 23']
merged_df['Total Funding'] = merged_df[funding_columns].sum(axis=1)
merged_df['Funding Per Student'] = merged_df['Total Funding'] / merged_df['Total Students 2021-2022']

# Round the columns
merged_df[funding_columns + ['Funding Per Student', 'Total Funding']] = merged_df[funding_columns + ['Funding Per Student', 'Total Funding']].round(2)

# Format dollar columns
format_dollar = lambda x: "${:,.0f}".format(x)
funding_dollar_columns = funding_columns + ['Funding Per Student', 'Total Funding']
renamed_columns = ['CARES ESSER DOLLAR', 'CRRSA ESSER DOLLAR', 'ARP ESSER DOLLAR', 'ESSER 22 DOLLAR', 'ESSER 23 DOLLAR', 'PER STUDENT DOLLAR', 'TOTAL DOLLAR']
merged_df[renamed_columns] = merged_df[funding_dollar_columns].applymap(format_dollar)

In [55]:
print(cdict)

{'Anderson': 48001, 'Andrews': 48003, 'Angelina': 48005, 'Aransas': 48007, 'Archer': 48009, 'Armstrong': 48011, 'Atascosa': 48013, 'Austin': 48015, 'Bailey': 48017, 'Bandera': 48019, 'Bastrop': 48021, 'Baylor': 48023, 'Bee': 48025, 'Bell': 48027, 'Bexar': 48029, 'Blanco': 48031, 'Borden': 48033, 'Bosque': 48035, 'Bowie': 48037, 'Brazoria': 48039, 'Brazos': 48041, 'Brewster': 48043, 'Briscoe': 48045, 'Brooks': 48047, 'Brown': 48049, 'Burleson': 48051, 'Burnet': 48053, 'Caldwell': 48055, 'Calhoun': 48057, 'Callahan': 48059, 'Cameron': 48061, 'Camp': 48063, 'Carson': 48065, 'Cass': 48067, 'Castro': 48069, 'Chambers': 48071, 'Cherokee': 48073, 'Childress': 48075, 'Clay': 48077, 'Cochran': 48079, 'Coke': 48081, 'Coleman': 48083, 'Collin': 48085, 'Collingsworth': 48087, 'Colorado': 48089, 'Comal': 48091, 'Comanche': 48093, 'Concho': 48095, 'Cooke': 48097, 'Coryell': 48099, 'Cottle': 48101, 'Crane': 48103, 'Crockett': 48105, 'Crosby': 48107, 'Culberson': 48109, 'Dallam': 48111, 'Dallas': 4811

In [56]:
#Creates a county map of Texas based off of the National Center on Education Statistics DataFrame
input_list = ['STAAR Score in Math',
              'STAAR Score in Reading',
              'Number of Students',
              'Number of Teachers',
              'Number of Staff'
             ]
funding_list = ['CARES ESSER I 20',
              'CRRSA ESSER II 21',
              'ARP ESSER III 21',
              'ESSER-SUPP 22',
              'ESSER-SUPP 23',
              'Funding Per Student'
              'Total Funding']
year_list = ['2019-2021','2021-2022','2019-2022','Funding']
district_list = label_data['County Name'].values
district_list = sorted(district_list)
class MapDashboard(param.Parameterized):
    

    __   = param.Selector(default='Hays',objects=district_list,check_on_set=False)
    ___  = param.Selector(default=year_list[0],objects=year_list)
    _    = param.ObjectSelector(default=input_list[0],objects=input_list)
    lists = {'2019-2021':['STAAR Score in Math','STAAR Score in Reading','Number of Students','Number of Teachers','Number of Staff'],
             '2021-2022':['STAAR Score in Math','STAAR Score in Reading','Number of Students','Number of Teachers','Number of Staff'],
             '2019-2022':['STAAR Score in Math','STAAR Score in Reading','Number of Students','Number of Teachers','Number of Staff'],
             'Funding':['CARES ESSER I 20','CRRSA ESSER II 21','ARP ESSER III 21','ESSER-SUPP 22','ESSER-SUPP 23','Funding Per Student','Total Funding']
            }
    @param.depends('___',watch=True)
    def update_list(self):
        up_list = self.lists[self.___]
        self.param['_'].objects = up_list
        self._ = up_list[0]
        
    def get_map(self):
        county_num = cdict[self.__]
        if self.___ == '2019-2021':
            yr_df = df_1921.loc[df_1921['County #'] == county_num]
            data = yr_df[['NAME','County #','DISTRICT_N',
                     #  'Label Math ','Label Reading ','Label Overall ', 
                       'Math%','Reading%','Loss%', self._]]
        elif self.___ == '2021-2022':
            yr_df = df_2122.loc[df_2122['County #'] == county_num]
            data = yr_df[['NAME','County #','DISTRICT_N',
                     #  'Label Math ','Label Reading ','Label Overall ', 
                       'Math%','Reading%','Loss%', self._]]
        elif self.___ == '2019-2022':
            yr_df = df_1922.loc[df_1922['County #'] == county_num]
            data = yr_df[['NAME','County #','DISTRICT_N',
                     #  'Label Math ','Label Reading ','Label Overall ', 
                       'Math%','Reading%','Loss%', self._]]
        elif self.___ == 'Funding':
            yr_df = merged_df.loc[merged_df['County #'] == county_num]
            data = yr_df[['NAME','County #','DISTRICT_N',
                          'CARES ESSER DOLLAR','CRRSA ESSER DOLLAR','ARP ESSER DOLLAR',
                          'ESSER 22 DOLLAR','ESSER 23 DOLLAR','PER STUDENT DOLLAR','TOTAL DOLLAR',self._]]
        
        dis_df = merged_df.loc[merged_df['County #'] == county_num]
        
        if self.___ == 'Funding':
            if self._ == 'CARES ESSER I 20':
                minimum = 0
                maximum = 2000000
            elif self._ == 'CRRSA ESSER II 21':
                minimum = 0
                maximum = 9000000
            elif self._ == 'ARP ESSER III 21':
                minimum = 0
                maximum = 20000000
            elif self._ == 'ESSER-SUPP 22':
                minimum = 0
                maximum = 200000
            elif self._ == 'ESSER-SUPP 23':
                minimum = 0
                maximum = 200000
            elif self._ == 'Funding Per Student':
                minimum = 0
                maximum = 6000
            else:
                minimum = 0
                maximum = 32000000
            fig = px.choropleth(data, geojson=dis_df, locations='DISTRICT_N', color= self._,
            color_continuous_scale='greens',
            range_color=(minimum,maximum),
            featureidkey='properties.DISTRICT_N',
            title =self.__ + ' County',
            custom_data = [data['NAME'],self._,
                           data['CARES ESSER DOLLAR'],data['CRRSA ESSER DOLLAR'],data['ARP ESSER DOLLAR'],
                          data['ESSER 22 DOLLAR'],data['ESSER 23 DOLLAR'],data['PER STUDENT DOLLAR'],data['TOTAL DOLLAR']],
            height=550, width=480)
            if self._ == 'CARES ESSER I 20':
                fig.update_layout(coloraxis_colorbar=dict(tickvals=[100000,550000,1000000,1450000,1900000],ticktext=['$0','$0.5M','$1M','$1.5M','$2M+']))
            elif self._ == 'CRRSA ESSER II 21':
                fig.update_layout(coloraxis_colorbar=dict(tickvals=[170000,2300000,4500000,6700000,8800000],ticktext=['$0','$2.25M','$4.5M','$6.75M','$9M+']))
            elif self._ == 'ARP ESSER III 21':
                fig.update_layout(coloraxis_colorbar=dict(tickvals=[1000000,5050000,10000000,14950000,19200000],ticktext=['$0','$5M','$10M','$15M','$20M+']))
            elif self._ == 'ESSER-SUPP 22':
                fig.update_layout(coloraxis_colorbar=dict(tickvals=[7000,50500,100000,149500,192000],ticktext=['$0','$50K','$100K','$150K','$200K+']))
            elif self._ == 'ESSER-SUPP 23':
                fig.update_layout(coloraxis_colorbar=dict(tickvals=[7000,50500,100000,149500,192000],ticktext=['$0','$50K','$100K','$150K','$200K+']))
            elif self._ == 'Funding Per Student':
                fig.update_layout(coloraxis_colorbar=dict(tickvals=[200,1550,3000,4450,5750],ticktext=['$0','$1500','$3000','$4500','$6000+']))
            elif self._ == 'Total Funding':
                fig.update_layout(coloraxis_colorbar=dict(tickvals=[1000000,8500000,16000000,23500000,31000000],ticktext=['$0','$8M','$16M','$24M','$32M+']))

        else:
            minimum = -10
            maximum = 10
            fig = px.choropleth(data, geojson=dis_df, locations='DISTRICT_N', color= self._,
            color_continuous_scale='balance_r',
            range_color=(minimum,maximum),
            featureidkey='properties.DISTRICT_N',
            title =self.__ + ' County',
            custom_data = [data['NAME'],self._,data['Math%'],data['Reading%'],data['Loss%']],
            height=550, width=480
                               )
            fig.update_layout(coloraxis_colorbar=dict(tickvals=[-8,-4,0,4,8],ticktext=['-10','-5','0','5','10']))
            
        fig.update_coloraxes(cmid=0,colorbar_orientation='h',colorbar_y=-0.1,colorbar_title_font_family='Times New Roman',
                             colorbar_title_font_size=22,colorbar_title_side='bottom',colorbar_thickness=4)
        if self.___ == 'Funding':
            if self._ == 'Total Funding':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '%{customdata[8]} in ' + self._
            elif self._ == 'CARES ESSER I 20':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '%{customdata[2]} from ' + self._
            elif self._ == 'CRRSA ESSER II 21':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '%{customdata[3]} from ' + self._
            elif self._ == 'ARP ESSER III 21':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '%{customdata[4]} from ' + self._
            elif self._ == 'ESSER-SUPP 22':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '%{customdata[5]} from ' + self._
            elif self._ == 'ESSER-SUPP 23':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '%{customdata[6]} from ' + self._
            elif self._ == 'Funding Per Student':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '%{customdata[7]} in ' + self._

        else:
            hovertemp = ''
            hovertemp += '%{customdata[0]}<br>'
            hovertemp += '%{customdata[1]}% change in ' + self._
        
            
        fig.update_traces(hovertemplate=hovertemp)
        fig.update_geos(fitbounds='locations',visible=False)
        fig.update_layout(hoverlabel=dict(bgcolor='black',font_size=24,font_family="Times New Roman"),
        font=dict(family='times new roman',size=23,color='black'),
        title_x=.5,
        title_y=.97,                  
        margin={"r":10,"t":0,"l":0,"b":0})
        return fig
        
    def get_description(self):
        descriptions = {
            "2019-2021": {
                "STAAR Score in Math": "The percent difference in <b>Math STAAR scores</b> between <b>2019 and 2021</b> for a district.",
                "STAAR Score in Reading": "The percent difference in <b>Reading STAAR scores</b> between <b>2019 and 2021</b> for a district.",
                "Number of Teachers": "The percent difference in <b>Full-Time Teachers</b> between <b>2019 and 2021</b> for a district.",
                "Number of Staff": "The percent difference in <b>staff</b> between <b>2019 and 2021</b> for a district.",
                "Number of Students": "The percent difference in <b>students</b> between <b>2019 and 2021</b> for a district."
            },
            "2021-2022": {
                "STAAR Score in Math": "The percent difference in <b>Math STAAR scores</b> between <b>2021 and 2022</b> for a district.",
                "STAAR Score in Reading": "The percent difference in <b>Reading STAAR scores</b> between <b>2021 and 2022</b> for a district.",
                "Number of Teachers": "The percent difference in <b>Full-Time Teachers</b> between <b>2021 and 2022</b> for a district.",
                "Number of Staff": "The percent difference in <b>staff</b> between <b>2021 and 2022</b> for a district.",
                "Number of Students": "The percent difference in <b>students</b> between <b>2021 and 2022</b> for a district."
            },
            "2019-2022": {
                "STAAR Score in Math": "The percent difference in <b>Math STAAR scores</b> between <b>2019 and 2022</b> for a district.",
                "STAAR Score in Reading": "The percent difference in <b>Reading STAAR scores</b> between <b>2019 and 2022</b> for a district.",
                "Number of Teachers": "The percent difference in <b>Full-Time Teachers</b> between <b>2019 and 2022</b> for a district.",
                "Number of Staff": "The percent difference in <b>staff</b> between <b>2019 and 2022</b> for a district.",
                "Number of Students": "The percent difference in <b>students</b> between <b>2019 and 2022</b> for a district."
            },
            "Funding": {
                "CARES ESSER I 20": "The dollar amount of funding granted to a district through the <b>CARES ESSER I 20 Program</b>.",
                "CRRSA ESSER II 21": "The dollar amount of funding granted to a district through the <b>CRRSA ESSER II 21 Program</b>.",
                "ARP ESSER III 21": "The dollar amount of funding granted to a district through the <b>ARP ESSER III 21 Program</b>.",
                "ESSER-SUPP 22": "The dollar amount of funding granted to a district through the <b>ESSER-SUPP 22 Program</b>.",
                "ESSER-SUPP 23": "The dollar amount of funding granted to a district through the <b>ESSER-SUPP 23 Program</b>.",
                "Funding Per Student": "The total dollar amount of funding per student granted to a district.",
                "Total Funding": "The total dollar amount of funding granted to a district."
            }
        }
        if self.___ in descriptions:
            description_text = descriptions[self.___].get(self._)
            if description_text:
                return pn.pane.HTML(description_text, 
                                    styles={'color':'black','font-size':'20px','width':'125%',
                                            'text-align':'center', 'max-width':'500px','word-wrap':'break-word',
                                            'margin-left': '147px', 'font-family':'Times New Roman'})
        return None
    
    
    def panel(self):
        return pn.Row(
            pn.Column(
            pn.pane.HTML("""<h1>Select A County & Variable To View At The District Level </h1>""",
                              styles={'color':'black','font-size':'14px','width':'100%',
                                     'text-align':'center',
                                     'font-family':'Times New Roman'}),
            
            # We're using a basic margin-left to center the widget in lieu the grid system due to ease of use, since this is a static dashboard.
            pn.Param(self,width=550,styles={'background': '#FFFFFF', 'margin-left': '213px'},
                     widgets={'__':{'widget_type':pn.widgets.AutocompleteInput,'placeholder':'Enter a county name...'},
                             '___':{'widget_type':pn.widgets.RadioButtonGroup},}
                    ),
            pn.Spacer(height=50),
            pn.pane.HTML("""<h1>What does this variable show us?</h1>""",
                            styles={'color':'black','font-size':'14px','width':'100%',
                                  'text-align':'center',
                                  'font-family':'Times New Roman'}),
            pn.Spacer(height=5),
            self.get_description,
            ),
            pn.Spacer(width=150),
            self.get_map
        )
dashboard = MapDashboard(name='')
map_component2 = dashboard.panel()
map_component2.save('DistrictLevelTab',resources=INLINE)

In [57]:
css = '''
.bk.panel-widget-box {
  background: #f0f0f0;
  border-radius: 5px;
  border: 1px black solid;
  font-size: 24px;
}

.title-bar {
  display: flex;
  justify-content: space-around;
  align-items: center;
  flex-wrap: wrap;
}

.title-html-pane {
  color: white;
  font-size: 20px;
  background-color: #6A5638;
  text-align: center;
  padding: 4px;
  font-family: Times New Roman;
  max-width: 800px;
  width: 100%;
  outline-style: solid;
  outline-color: #30231D;
  outline-width: thick;
  display: flex;
}

.image-box {
  display: flex;
  justify-content: center;
  align-items: center;
  background-color: white;
  padding: 4px;
}


}
'''

pn.extension(raw_css=[css])

# County Level Dashboard

In [58]:
# Using CSS and HTML to manually align text since gridspec was sending it all over the place. 
# All of these could likely be improved by digging through the panel documentation.
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                               css_classes=['title-html-pane'],
                               styles={'color':'white','font-size':'14px','width':'100%',
                                       'background-color':'#6A5638','text-align':'center',
                                       'outline-style':'solid','outline-color':'#30231D',
                                       'outline-width':'thick','font-family':'Times New Roman',
                                       'height':'140px','padding':'10px','border-radius':'10px'})

datalab_img = pn.pane.PNG("Data/Images/datalabimage.png", height=140, width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png", height=140, width=350)

image_row = pn.Row(
    datalab_img,
    title_html_pane,
    txst_img,
    css_classes=['image-box']
)

content_bar = pn.Row(map_component1)
app_layout = pn.Column(image_row, content_bar)
app_layout.show()

Launching server at http://localhost:51855


# District Level Dashboard

In [59]:
#Panel HTML elements can take a 'styles' parameter directly. 
#We are alternatively assigning a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                               css_classes=['title-html-pane'],
                               styles={'color':'white','font-size':'14px','width':'100%',
                                       'background-color':'#6A5638','text-align':'center',
                                       'outline-style':'solid','outline-color':'#30231D',
                                       'outline-width':'thick','font-family':'Times New Roman',
                                       'height':'140px','padding':'10px','border-radius':'10px'})

datalab_img = pn.pane.PNG("Data/Images/datalabimage.png", height=140, width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png", height=140, width=350)

image_row = pn.Row(
    datalab_img,
    title_html_pane,
    txst_img,
    css_classes=['image-box']
)

content_bar = pn.Row(map_component2)
app_layout = pn.Column(image_row, content_bar)
app_layout.show()

Launching server at http://localhost:51856


# Heatmap & Histogram Dashboard

In [60]:
#Panel HTML elements can take a 'styles' parameter directly. 
#We are alternatively assigning a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                               css_classes=['title-html-pane'],
                               styles={'color':'white','font-size':'14px','width':'100%',
                                       'background-color':'#6A5638','text-align':'center',
                                       'outline-style':'solid','outline-color':'#30231D',
                                       'outline-width':'thick','font-family':'Times New Roman',
                                       'height':'140px','padding':'10px','border-radius':'10px'})

datalab_img = pn.pane.PNG("Data/Images/datalabimage.png", height=140, width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png", height=140, width=350)

image_row = pn.Row(
    datalab_img,
    title_html_pane,
    txst_img,
    css_classes=['image-box']
)

content_bar = pn.Row(loss_component1,loss_component2)
app_layout = pn.Column(image_row, content_bar)
app_layout.show()

Launching server at http://localhost:51858


# Definitions

In [61]:
#Panel HTML elements can take a 'styles' parameter directly. 
#We are alternatively assigning a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                               css_classes=['title-html-pane'],
                               styles={'color':'white','font-size':'14px','width':'100%',
                                       'background-color':'#6A5638','text-align':'center',
                                       'font-family':'Times New Roman','height':'140px',
                                       'padding':'10px','border-radius':'10px'})

datalab_img = pn.pane.PNG("Data/Images/datalabimage.png", height=140, width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png", height=140, width=350)

image_row = pn.Row(
    datalab_img,
    title_html_pane,
    txst_img,
    css_classes=['image-box']
)

# Manually modifying the HTML/CSS for compatibilities sake
index_pane = pn.pane.HTML(
    """<h1 style='margin-left: 311px;'>Variable Definitions:</h1>""",
    styles={
        'color': 'white',
        'font-size': '20px',
        'width': '100%',
        'margin-left': '300px',
        'outline-style': 'solid',
        'outline-color': '#30231D',
        'outline-width': 'thin',
        'background-color': '#501214',
        'font-family': 'Times New Roman',
        'border': '4px solid black',
        'padding': '5px',
        'border-radius': '5px'
    }
)

table = pn.pane.HTML("""
    <table style="margin: 0 auto; width: 100%; font-size: 16px; text-align: left;">
        <tr>
            <th>Variable</th>
            <th>Reflects the percent difference of...</th>
        </tr>
        <tr>
            <td>STAAR Score in Math</td>
            <td>Math STAAR scores between X and Y years for a district/county.</td>
        </tr>
        <tr>
            <td>STAAR Score in Reading</td>
            <td>Reading STAAR scores between X and Y years for a district/county.</td>
        </tr>
        <tr>
            <td>Labor Force</td>
            <td>Labor force between X and Y years for a county.</td>
        </tr>
        <tr>
            <td>Number of Overall Staff</td>
            <td>Staff between X and Y years for a district/county.</td>
        </tr>
        <tr>
            <td>Number of Students</td>
            <td>Students between X and Y years for a district/county.</td>
        </tr>
        <tr>
            <td>Number of Teachers</td>
            <td>Full-Time Teachers between X and Y years for a district/county.</td>
        </tr>
    </table>
""", styles={
    'width': '100%',
    'font-size': '16px',
    'text-align': 'left',
    'font-family': 'Times New Roman',
    'border': '4px solid black',
    'padding': '10px',
    'border-radius': '10px',
    'margin-left':'300px'
})

content_bar = pn.Column(index_pane, table)
app_layout = pn.Column(image_row, content_bar)
app_layout.show()


Launching server at http://localhost:51863
